### Code để crawl id của từng bds trên trang, limit mỗi page lấy 20 id, chỉnh số page trong phần for page in range(30):
Tự đổi path_file cho phù hợp

In [3]:
import requests
import json
import time
id_path_file = r"D:\\20251\\Lưu trữ và xử lý dữ liệu lớn\\project\\data\\all_ids.json"
path_raw_data = r"D:\\20251\\Lưu trữ và xử lý dữ liệu lớn\\project\\data\\all_raw_data.json"
import json

In [ ]:


try:
    with open(id_path_file, "r", encoding="utf-8") as f:
        all_ids = set(json.load(f))  # dùng set để tránh trùng lặp
except FileNotFoundError:
    all_ids = set()

print(f"Đã load {len(all_ids)} ID từ file all_ids.json")
def get_ad_ids(page, limit=20):
    offset = page * limit
    url = f"https://gateway.chotot.com/v1/public/ad-listing?region_v2=12000&cg=1000&o={offset}&limit={limit}"
    headers = {"User-Agent": "Mozilla/5.0"}
    resp = requests.get(url, headers=headers, timeout=10)
    resp.raise_for_status()
    data = resp.json()
    ads = data.get("ads", [])
    ad_ids = [ad["list_id"] for ad in ads if "list_id" in ad]
    return ad_ids



for page in range(30,60):  # crawl 5 trang đầu tiên
    ids = get_ad_ids(page)
    before = len(all_ids)
    all_ids.update(ids)  # thêm và tự loại trùng
    print(f"Trang {page+1}: lấy được {len(ids)} ID, thêm mới {len(all_ids) - before}")
    time.sleep(2)  # tránh bị chặn


all_ids_list = list(all_ids)  # nếu là set, convert sang list trước khi lưu

# Lưu ra file JSON
with open(id_path_file, "w", encoding="utf-8") as f:
    json.dump(all_ids_list, f, ensure_ascii=False, indent=2)

print(f"Đã lưu {len(all_ids_list)} ID vào file all_ids.json")
print("Tổng cộng:", len(all_ids), "ad_id duy nhất")
print("Một vài ID đầu tiên:", list(all_ids)[:10])


Đã load 618 ID từ file all_ids.json
Trang 31: lấy được 20 ID, thêm mới 20
Trang 32: lấy được 20 ID, thêm mới 20
Trang 33: lấy được 20 ID, thêm mới 20
Trang 34: lấy được 20 ID, thêm mới 19
Trang 35: lấy được 20 ID, thêm mới 19
Trang 36: lấy được 20 ID, thêm mới 20
Trang 37: lấy được 20 ID, thêm mới 20
Trang 38: lấy được 20 ID, thêm mới 20
Trang 39: lấy được 20 ID, thêm mới 20
Trang 40: lấy được 20 ID, thêm mới 19
Trang 41: lấy được 20 ID, thêm mới 20
Trang 42: lấy được 20 ID, thêm mới 20
Trang 43: lấy được 20 ID, thêm mới 20
Trang 44: lấy được 20 ID, thêm mới 20
Trang 45: lấy được 20 ID, thêm mới 20
Trang 46: lấy được 20 ID, thêm mới 20
Trang 47: lấy được 20 ID, thêm mới 20
Trang 48: lấy được 20 ID, thêm mới 20
Trang 49: lấy được 20 ID, thêm mới 20
Trang 50: lấy được 20 ID, thêm mới 19
Trang 51: lấy được 20 ID, thêm mới 20
Trang 52: lấy được 20 ID, thêm mới 19
Trang 53: lấy được 20 ID, thêm mới 20
Trang 54: lấy được 20 ID, thêm mới 19
Trang 55: lấy được 20 ID, thêm mới 20
Trang 56: lấy 

### Code crawl thông tin chi tiết của 1 post dựa vào id đã crawl bên trên

In [16]:
import requests
import json
def crawl_data(ad_id, debug = False):
    
    url = f"https://gateway.chotot.com/v1/public/ad-listing/{ad_id}"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
        "Accept": "application/json",
    }

    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    data = resp.json()

    # In đẹp JSON
    if debug==True:
        print(json.dumps(data, indent=2, ensure_ascii=False))
        # Lấy các field quan trọng
        ad = data.get("ad")
        print(type(data))
        print("Tiêu đề:", ad.get("subject"))
        print("Giá:", ad.get("price"))
        print("Diện tích:", ad.get("area"))
        print("Địa chỉ:", ad.get("region_name"), "-", ad.get("area_name"))
        print("Mô tả:", ad.get("body"))
    return data

crawl_data("128033818")

HTTPError: 404 Client Error: Not Found for url: https://gateway.chotot.com/v1/public/ad-listing/128033818

In [17]:
try:
    with open(id_path_file, "r", encoding="utf-8") as f:
        all_ids = set(json.load(f))
except FileNotFoundError:
    all_ids = set()

# --- Đọc dữ liệu đã crawl ---
try:
    with open(path_raw_data, "r", encoding="utf-8") as f:
        all_raw_data = json.load(f)
except FileNotFoundError:
    all_raw_data = {}

crawled_id = set(all_raw_data.keys())
print("Đã crawl", len(crawled_id), "id")

uncrawled_ids = [id for id in all_ids if str(id) not in crawled_id]
print("Còn lại", len(uncrawled_ids), "id chưa crawl.")

for i, id in enumerate(uncrawled_ids):
    try:
        raw_data = crawl_data(str(id))  # hàm crawl_data của bạn
        if raw_data is None:
            raise ValueError("Không có dữ liệu trả về")  # để xử lý trường hợp dữ liệu rỗng
        all_raw_data[str(id)] = raw_data

    except Exception as e:
        # Nếu bị lỗi 404 hoặc lỗi nào khác thì bỏ qua id này
        err_msg = str(e).lower()
        if "404" in err_msg or "not found" in err_msg:
            print(f"❌ ID {id} bị lỗi 404 — xóa khỏi danh sách và bỏ qua.")
            all_ids.discard(id)
            with open(id_path_file, "w", encoding="utf-8") as f:
                json.dump(list(all_ids), f, ensure_ascii=False, indent=2)
                f.flush()
        else:
            print(f"⚠️ Lỗi khi crawl ID {id}: {e}")
        continue

    # Lưu định kỳ mỗi 100 ID
    if i % 100 == 0 and i > 0:
        with open(path_raw_data, "w", encoding="utf-8") as f:
            json.dump(all_raw_data, f, ensure_ascii=False, indent=2)
            f.flush()
        print(f"Đã lưu {len(all_raw_data)} bản ghi.")

    time.sleep(1)  # tránh bị chặn IP

# --- Lưu lần cuối ---
with open(path_raw_data, "w", encoding="utf-8") as f:
    json.dump(all_raw_data, f, ensure_ascii=False, indent=2)
    f.flush()

print("✅ Crawl hoàn tất. Tổng cộng lưu được", len(all_raw_data), "bản ghi.")

Đã crawl 503 id
Còn lại 688 id chưa crawl.
❌ ID 128033818 bị lỗi 404 — xóa khỏi danh sách và bỏ qua.
Đã lưu 603 bản ghi.
Đã lưu 703 bản ghi.
Đã lưu 803 bản ghi.
❌ ID 128507374 bị lỗi 404 — xóa khỏi danh sách và bỏ qua.
Đã lưu 902 bản ghi.
Đã lưu 1002 bản ghi.
Đã lưu 1102 bản ghi.
❌ ID 128511933 bị lỗi 404 — xóa khỏi danh sách và bỏ qua.
✅ Crawl hoàn tất. Tổng cộng lưu được 1188 bản ghi.


In [14]:
print(all_ids)

{121993216, 128518145, 126779393, 128524301, 128520206, 128514066, 128368658, 128522260, 128442391, 128507930, 128499738, 128514074, 128514079, 127635488, 127586338, 128245794, 128501794, 128014373, 127068203, 127885355, 127393841, 127486003, 128520245, 127723574, 127012920, 128493625, 128516153, 128159803, 128524348, 126427196, 128507968, 128520258, 128481347, 128518212, 124162117, 128262218, 128510030, 127510606, 128174160, 128510032, 127592529, 128520275, 120557649, 128311381, 128514134, 128514135, 127658072, 128514137, 128522330, 126543960, 127635547, 128086111, 126617695, 128520290, 128464995, 128516195, 127422567, 128458857, 128487529, 128432235, 128514156, 127357037, 128520302, 128514159, 128510065, 128508018, 128493686, 128522360, 128444538, 128149628, 128481405, 128161917, 128481406, 122927232, 128030847, 124688515, 128514182, 128155783, 123494536, 128518278, 126232714, 127131787, 127262859, 128520331, 127531163, 128514210, 127658146, 128514213, 123091110, 128516261, 128512168

In [3]:
import os
import os

os.environ["JAVA_HOME"] = "C:\\Program Files\\Eclipse Adoptium\\jdk-11.0.28"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "\\bin;" + os.environ["PATH"]
print(os.environ.get("JAVA_HOME"))
print(os.popen("java -version").read())


C:\Program Files\Eclipse Adoptium\jdk-11.0.28



In [4]:
import json

with open(r"D:\\20251\\Lưu trữ và xử lý dữ liệu lớn\\project\\github\\BigData_Real_Estate_Group13\\streaming_input\\all_raw_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

with open("data_lines.json", "w", encoding="utf-8") as f:
    for key, value in data.items():
        obj = {"id": key, "data": value}
        f.write(json.dumps(obj, ensure_ascii=False) + "\n")


In [3]:
import findspark
findspark.init()
findspark.find()

'E:\\miniconda\\envs\\spark_env\\Lib\\site-packages\\pyspark'

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StringType, IntegerType

import os

# Tắt Hadoop native libraries
os.environ['HADOOP_HOME'] = ''
os.environ['HADOOP_CONF_DIR'] = ''

# Hoặc set property này
spark = SparkSession.builder \
    .appName("StructuredStreamingExample") \
    .master("local[*]") \
    .config("spark.hadoop.io.nativeio.windows", "false") \
    .getOrCreate()

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, LongType, IntegerType, BooleanType
from pyspark.sql.functions import col, from_json

spark = SparkSession.builder \
    .appName("RealEstateJSONtoHDFS") \
    .master("local[*]") \
    .getOrCreate()

# Định nghĩa schema JSON
ad_schema = StructType([
    StructField("ad_id", LongType(), True),
    StructField("list_id", LongType(), True),
    StructField("list_time", LongType(), True),
    StructField("state", StringType(), True),
    StructField("type", StringType(), True),
    StructField("account_name", StringType(), True),
    StructField("phone", StringType(), True),
    StructField("region", IntegerType(), True),
    StructField("category", IntegerType(), True),
    StructField("company_ad", BooleanType(), True),
    StructField("subject", StringType(), True),
    StructField("body", StringType(), True)
])

data_schema = StructType([StructField("ad", ad_schema, True)])
main_schema = StructType([StructField("id", StringType(), True),
                          StructField("data", data_schema, True)])

# Thư mục chứa JSON streaming
input_path = r"D:\\20251\\Lưu trữ và xử lý dữ liệu lớn\\project\\github\\BigData_Real_Estate_Group13\\streaming_input"
input_path = input_path.replace("\\","/")

stream_df = spark.readStream \
    .schema(main_schema) \
    .option("maxFilesPerTrigger", 1) \
    .json(input_path)

# Parse JSON nested
processed_df = stream_df.select(
    col("id").alias("list_id"),
    col("data.ad.*")
)
processed_df.writeStream \
    .format("parquet") \
    .option("path", "hdfs://localhost:9000/streaming/realestate") \
    .option("checkpointLocation", "hdfs://localhost:9000/checkpoints/realestate") \
    .outputMode("append") \
    .start() \
    .awaitTermination()


Py4JJavaError: An error occurred while calling o50.start.
: java.net.ConnectException: Call From DESKTOP-17NN6TB/172.20.10.2 to localhost:9000 failed on connection exception: java.net.ConnectException: Connection refused: getsockopt; For more details see:  http://wiki.apache.org/hadoop/ConnectionRefused
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:75)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:53)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:502)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:486)
	at org.apache.hadoop.net.NetUtils.wrapWithMessage(NetUtils.java:961)
	at org.apache.hadoop.net.NetUtils.wrapException(NetUtils.java:876)
	at org.apache.hadoop.ipc.Client.getRpcResponse(Cl

In [5]:
spark

In [4]:
import pyspark
print(pyspark.__version__)  # Ví dụ: 3.5.0

4.0.1


In [3]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, IntegerType



# Đường dẫn đến thư mục chứa file JSON
input_path = r"D:\\20251\\Lưu trữ và xử lý dữ liệu lớn\\project\\github\\BigData_Real_Estate_Group13\\streaming_input"  # Thay đổi đường dẫn này

# Định nghĩa schema
schema = StructType() \
    .add("id", IntegerType()) \
    .add("data", StructType())

# Đọc luồng dữ liệu
stream_df = spark.readStream \
    .schema(schema) \
    .json(input_path)

# Hiển thị dữ liệu ra console
query = stream_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

# Chờ query kết thúc
query.awaitTermination()

# Hoặc chạy trong khoảng thời gian giới hạn
# query.awaitTermination(timeout=60)  # 60 giây

Py4JJavaError: An error occurred while calling o34.json.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:817)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1415)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1620)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:739)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2078)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2122)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:961)
	at org.apache.spark.util.HadoopFSUtils$.listLeafFiles(HadoopFSUtils.scala:218)
	at org.apache.spark.util.HadoopFSUtils$.$anonfun$parallelListLeafFilesInternal$1(HadoopFSUtils.scala:132)
	at scala.collection.immutable.List.map(List.

In [6]:


# ... rest of your code ...
# Đọc dữ liệu streaming từ folder
input_path = "D:/20251/Lưu trữ và xử lý dữ liệu lớn/project/github/BigData_Real_Estate_Group13/streaming_input"

schema = StructType() \
    .add("id", IntegerType()) \
    .add("data", StructType())

# Đọc luồng dữ liệu - This should now pass the native check
stream_df = spark.readStream \
    .schema(schema) \
    .json(input_path)

# Hiển thị dữ liệu ra console
query = stream_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination()

Py4JJavaError: An error occurred while calling o46.json.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:817)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1415)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1620)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:739)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2078)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2122)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:961)
	at org.apache.spark.util.HadoopFSUtils$.listLeafFiles(HadoopFSUtils.scala:218)
	at org.apache.spark.util.HadoopFSUtils$.$anonfun$parallelListLeafFilesInternal$1(HadoopFSUtils.scala:132)
	at scala.collection.immutable.List.map(List.